In [6]:
import tensorflow as tf
import os
from keras.preprocessing import image
import matplotlib.pyplot as plt 
import numpy as np
from keras.utils.np_utils import to_categorical
import random,shutil
from keras.models import Sequential
from keras.layers import Dropout,Conv2D,Flatten,Dense, MaxPooling2D, BatchNormalization
from keras.models import load_model


In [7]:
def generator(dir, gen=image.ImageDataGenerator(rescale=1./255), 
              shuffle=True,batch_size=1,
              target_size=(24,24),
              class_mode='categorical'):
    
    return gen.flow_from_directory(dir,batch_size=batch_size,
                                   shuffle=shuffle,color_mode='grayscale',
                                   class_mode=class_mode,
                                   target_size=target_size)

In [8]:
BS= 32
TS=(24,24)
train_batch= generator('Datasets\\Drowsiness_Detection_Dataset\\dataset_new\\train',shuffle=True, batch_size=BS,target_size=TS)
valid_batch= generator('Datasets\\Drowsiness_Detection_Dataset\\dataset_new\\test',shuffle=True, batch_size=BS,target_size=TS)
SPE= len(train_batch.classes)//BS
VS = len(valid_batch.classes)//BS
print(SPE,VS)

Found 2467 images belonging to 4 classes.
Found 433 images belonging to 4 classes.
77 13


In [9]:
# img,labels= next(train_batch)
# print(img.shape)

model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24,24,1)),
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
#32 convolution filters used each of size 3x3
#again
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(1,1)),

#64 convolution filters used each of size 3x3
#choose the best features via pooling
    
#randomly turn neurons on and off to improve convergence
    Dropout(0.25),
#flatten since too many dimensions, we only want a classification output
    Flatten(),
#fully connected to get all relevant data
    Dense(128, activation='relu'),
#one more dropout for convergence' sake :) 
    Dropout(0.5),
#output a softmax to squash the matrix into output probabilities
    Dense(4, activation='softmax')
])


In [10]:
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

train_model = model.fit(train_batch, 
                                  validation_data=valid_batch,
                                  epochs=15,
                                  steps_per_epoch=SPE,
                                  validation_steps=VS)
model.save('drowsiness_detection.h5', overwrite = True)

Epoch 1/15
77/77 [==============================] - 8s 98ms/step - loss: 0.6690 - accuracy: 0.6854 - val_loss: 0.4436 - val_accuracy: 0.7981
Epoch 2/15
77/77 [==============================] - 7s 87ms/step - loss: 0.4008 - accuracy: 0.8057 - val_loss: 0.3483 - val_accuracy: 0.8221
Epoch 3/15
77/77 [==============================] - 7s 86ms/step - loss: 0.3423 - accuracy: 0.8292 - val_loss: 0.3421 - val_accuracy: 0.8486
Epoch 4/15
77/77 [==============================] - 6s 84ms/step - loss: 0.3050 - accuracy: 0.8542 - val_loss: 0.3078 - val_accuracy: 0.8678
Epoch 5/15
77/77 [==============================] - 6s 82ms/step - loss: 0.2726 - accuracy: 0.8702 - val_loss: 0.2944 - val_accuracy: 0.8558
Epoch 6/15
77/77 [==============================] - 6s 84ms/step - loss: 0.2593 - accuracy: 0.8896 - val_loss: 0.2745 - val_accuracy: 0.8654
Epoch 7/15
77/77 [==============================] - 6s 82ms/step - loss: 0.2177 - accuracy: 0.9014 - val_loss: 0.2581 - val_accuracy: 0.8846
Epoch 8/15
77